# Day 2, Block A: SQL Joins & Relational Modeling

**Duration:** 60 minutes  
**Course:** ECBS5294 - Introduction to Data Science: Working with Data  
**Instructor:** Eduardo Ariño de la Rubia

---

## Learning Objectives

By the end of this session, you will be able to:

1. **Explain** primary keys and foreign keys in relational databases
2. **Read** basic ERDs (entity-relationship diagrams)
3. **Write INNER JOIN queries** to combine matching records
4. **Write LEFT JOIN queries** to keep all records from one table
5. **Use CTEs** (Common Table Expressions) for readable multi-step queries
6. **Recognize duplicate inflation** and avoid it
7. **Aggregate correctly** after joins with proper GROUP BY
8. **Combine** joins with window functions for advanced analytics

---

## Part 1: Setup & Motivation (⏱️ 5 minutes)

### Why Joins Matter

> **"When you connect tables, you're connecting different aspects of the business. Joins are how organizations answer complex questions."**

**The Business Reality:**
- Your data lives in **multiple tables**, not one giant spreadsheet
- **Customers** place **orders**
- **Orders** contain **products**
- **Products** come from **sellers**
- Customers leave **reviews** about orders

**To answer business questions, you need to JOIN these tables together:**
- "Which products generate the most revenue?" → Join orders + products
- "Which sellers have the best ratings?" → Join sellers + reviews
- "What percentage of orders lack reviews?" → Join orders + reviews (carefully!)

**Today's dataset:** Brazilian e-commerce marketplace (Olist) with real customer, product, and seller data.

---

### Setup: Connect to DuckDB and Load Data

In [1]:
# Import libraries
import duckdb
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Connect to in-memory database
con = duckdb.connect(':memory:')

print("✅ Connected to DuckDB!")
print(f"DuckDB version: {duckdb.__version__}")

✅ Connected to DuckDB!
DuckDB version: 1.4.1


### Load Olist Tables

We'll load the core tables we need for today's session:

In [2]:
# Load orders (fact table)
con.execute("""
    CREATE TABLE orders AS
    SELECT * FROM '../../data/day2/block_a/olist_orders_dataset.csv'
""")

# Load customers (dimension table)
con.execute("""
    CREATE TABLE customers AS
    SELECT * FROM '../../data/day2/block_a/olist_customers_dataset.csv'
""")

# Load order_items (fact table)
con.execute("""
    CREATE TABLE order_items AS
    SELECT * FROM '../../data/day2/block_a/olist_order_items_dataset.csv'
""")

# Load products (dimension table)
con.execute("""
    CREATE TABLE products AS
    SELECT * FROM '../../data/day2/block_a/olist_products_dataset.csv'
""")

# Load sellers (dimension table)
con.execute("""
    CREATE TABLE sellers AS
    SELECT * FROM '../../data/day2/block_a/olist_sellers_dataset.csv'
""")

# Load reviews (for LEFT JOIN examples)
con.execute("""
    CREATE TABLE reviews AS
    SELECT * FROM '../../data/day2/block_a/olist_order_reviews_dataset.csv'
""")

# Load category translation (for readable category names)
con.execute("""
    CREATE TABLE categories AS
    SELECT * FROM '../../data/day2/block_a/product_category_name_translation.csv'
""")

print("✅ All tables loaded successfully!")

✅ All tables loaded successfully!


### Verify Table Sizes

In [3]:
# Check row counts
con.execute("""
    SELECT 'orders' as table_name, COUNT(*) as row_count FROM orders
    UNION ALL
    SELECT 'customers', COUNT(*) FROM customers
    UNION ALL
    SELECT 'order_items', COUNT(*) FROM order_items
    UNION ALL
    SELECT 'products', COUNT(*) FROM products
    UNION ALL
    SELECT 'sellers', COUNT(*) FROM sellers
    UNION ALL
    SELECT 'reviews', COUNT(*) FROM reviews
    ORDER BY row_count DESC
""").df()

,table_name,row_count
0,order_items,112650
1,orders,99441
2,customers,99441
3,reviews,99224
4,products,32951
5,sellers,3095


**Key observations:**
- **99,441 orders** and **99,441 customers** → One customer per order (by design)
- **112,650 order_items** → More items than orders (some orders have multiple items!)
- **32,951 products** → Large catalog
- **3,095 sellers** → Multi-seller marketplace
- **~100K reviews** → Most orders have reviews, but not all

---

In [5]:
# Verify join key types match across tables
con.execute("""
    SELECT 
        'orders.customer_id' as column_name,
        typeof((SELECT customer_id FROM orders LIMIT 1)) as data_type
    UNION ALL
    SELECT 
        'customers.customer_id',
        typeof((SELECT customer_id FROM customers LIMIT 1))
    UNION ALL
    SELECT 
        'orders.order_id',
        typeof((SELECT order_id FROM orders LIMIT 1))
    UNION ALL
    SELECT 
        'order_items.order_id',
        typeof((SELECT order_id FROM order_items LIMIT 1))
""").df()

,column_name,data_type
0,orders.customer_id,VARCHAR
1,customers.customer_id,VARCHAR
2,orders.order_id,VARCHAR
3,order_items.order_id,VARCHAR


**Perfect!** All join keys have matching types (VARCHAR). If types didn't match (e.g., one table had INTEGER and another had VARCHAR), joins would fail with type errors.

**This is why you always check types before joining tables.**

---

**Key observations:**
- ✅ `order_id` and `customer_id` are **VARCHAR** (strings) - These are hash-based IDs, not sequential numbers!
- ✅ Timestamps correctly inferred as **TIMESTAMP** (not text!)
- ✅ Status is **VARCHAR** (categorical text)

**Why are IDs strings?** Modern systems often use UUIDs or hash-based IDs (like `e481f51cbdc54678b7cc49136f2d6af7`) instead of auto-incrementing integers. This prevents:
- Exposing business metrics (e.g., "we have exactly 100,000 orders")
- ID conflicts when merging databases
- Security issues from predictable IDs

**Pro tip:** In production, you'd explicitly specify types in `CREATE TABLE`. But DuckDB's inference is quite good - just always verify!

**Now let's verify that join keys have matching types across tables:**

In [6]:
# Check inferred types for orders table
con.execute("DESCRIBE orders").df()

,column_name,column_type,null,key,default,extra
0,order_id,VARCHAR,YES,None,None,None
1,customer_id,VARCHAR,YES,None,None,None
2,order_status,VARCHAR,YES,None,None,None
3,order_purchase_timestamp,TIMESTAMP,YES,None,None,None
4,order_approved_at,TIMESTAMP,YES,None,None,None
5,order_delivered_carrier_date,TIMESTAMP,YES,None,None,None
6,order_delivered_customer_date,TIMESTAMP,YES,None,None,None
7,order_estimated_delivery_date,TIMESTAMP,YES,None,None,None


### Verify Data Types

**Always check what types were inferred!**

DuckDB automatically infers column types from CSV files. Let's verify it understood our data correctly by looking at the full table structure:

### Understanding the Schema: ERD (Entity-Relationship Diagram)

```
┌─────────────────┐
│   customers     │
│  (99,441)       │
│                 │
│ customer_id (PK)│───┐
└─────────────────┘   │
                      │
                      │
┌─────────────────┐   │    ┌──────────────────┐
│   orders        │   │    │  order_items     │
│  (99,441)       │   │    │  (112,650)       │
│                 │   │    │                  │
│ order_id (PK)   │◄──┼────│ order_id (FK)    │
│ customer_id(FK) │◄──┘    │ product_id (FK)  │───┐
└─────────────────┘        │ seller_id (FK)   │───┼───┐
       ▲                   └──────────────────┘   │   │
       │                                          │   │
       │                   ┌──────────────────┐   │   │
       │                   │   products       │   │   │
       │                   │  (32,951)        │   │   │
       │                   │                  │   │   │
       │                   │ product_id (PK)  │◄──┘   │
       │                   └──────────────────┘       │
       │                                              │
       │                   ┌──────────────────┐       │
       │                   │   sellers        │       │
       │                   │  (3,095)         │       │
       │                   │                  │       │
       │                   │ seller_id (PK)   │◄──────┘
       │                   └──────────────────┘
       │
       │
┌──────▼──────────┐
│   reviews       │
│  (~100K)        │
│                 │
│ order_id (FK)   │
└─────────────────┘
```

**Key concepts:**
- **PK (Primary Key):** Uniquely identifies each row in a table
- **FK (Foreign Key):** Points to a primary key in another table (creates the relationship)
- **One-to-many:** One customer → many orders, one order → many items

---

### Primary Keys vs Foreign Keys

> **Primary Key (PK):** "This column uniquely identifies each row in THIS table"
> 
> **Foreign Key (FK):** "This column points to a primary key in ANOTHER table"

**Example:**
- `orders.order_id` is a **primary key** (each order has unique ID)
- `orders.customer_id` is a **foreign key** (points to `customers.customer_id`)
- `order_items.order_id` is a **foreign key** (points to `orders.order_id`)

**Why this matters:**
- Foreign keys tell you **how to join tables**
- Join condition: `orders.customer_id = customers.customer_id`
- This creates the "connection" between tables

Let's explore the data before joining:

In [8]:
# Look at orders table
con.execute("""
    SELECT 
        order_id,
        customer_id,
        order_status,
        order_purchase_timestamp
    FROM orders
    LIMIT 5
""").df()

,order_id,customer_id,order_status,order_purchase_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39


In [9]:
# Look at customers table
con.execute("""
    SELECT 
        customer_id,
        customer_city,
        customer_state
    FROM customers
    LIMIT 5
""").df()

,customer_id,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,campinas,SP


**Notice:** `customer_id` appears in BOTH tables. That's our join key!

---

## Part 2: The Core - Join Types (⏱️ 27 minutes)

### INNER JOIN: Only Matching Records (⏱️ 12 minutes)

> **INNER JOIN returns rows that have matches in BOTH tables**

![INNER JOIN Venn Diagram](../../references/images/inner_join.svg)

**Business use case:** "Show me orders WITH customer details (only completed matches)"

---


### Example 1: Basic INNER JOIN

**Business question:** "Show me orders with customer location information"

**Query structure:**
```sql
SELECT columns
FROM table1
INNER JOIN table2 ON table1.key = table2.key
```

In [10]:
# Join orders with customers
con.execute("""
    SELECT 
        o.order_id,
        o.order_status,
        o.order_purchase_timestamp,
        c.customer_city,
        c.customer_state
    FROM orders o
    INNER JOIN customers c ON o.customer_id = c.customer_id
    LIMIT 10
""").df()

,order_id,order_status,order_purchase_timestamp,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,vianopolis,GO
3,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,santo andre,SP
4,a4591c265e18cb1dcee52889e2d8acc3,delivered,2017-07-09 21:57:05,congonhinhas,PR
5,136cce7faa42fdb2cefd53fdc79a6098,invoiced,2017-04-11 12:22:08,santa rosa,RS
6,6514b8ad8028c9f2cc2374ded245783f,delivered,2017-05-16 13:10:30,nilopolis,RJ
7,76c6e866289321a7c93b82b54852dc33,delivered,2017-01-23 18:29:09,faxinalzinho,RS
8,e69bfb5eb88e0ed6a785585b27e16dbf,delivered,2017-07-29 11:55:02,sorocaba,SP
9,e6ce16cb79ec1d90b1da9085a6118aeb,delivered,2017-05-16 19:41:10,rio de janeiro,RJ


**What happened:**
1. DuckDB looked at each `order_id` in the `orders` table
2. Found the matching `customer_id` in the `customers` table
3. Combined columns from both tables into one result
4. **Only returned rows where a match existed**

**Table aliases:** `o` and `c` are shorthand for `orders` and `customers` (makes queries readable)

---

### Example 2: INNER JOIN with Aggregation

**Business question:** "How many orders came from each state?"

In [17]:
# Count orders by customer state
con.execute("""
    SELECT 
        c.customer_state,
        COUNT(o.order_id) AS order_count
    FROM orders o
    INNER JOIN customers c ON o.customer_id = c.customer_id
    GROUP BY c.customer_state
    ORDER BY order_count DESC
    LIMIT 10
""").df()

,customer_state,order_count
0,SP,41746
1,RJ,12852
2,MG,11635
3,RS,5466
4,PR,5045
5,SC,3637
6,BA,3380
7,DF,2140
8,ES,2033
9,GO,2020


**Insight:** São Paulo (SP) dominates with the most orders. This is Brazil's economic center.

---

### Example 3: Multi-table INNER JOIN

**Business question:** "Show me revenue by product category (in English)"

This requires joining **three tables:**
1. `order_items` (has price and product_id)
2. `products` (has category name in Portuguese)
3. `categories` (translates category to English)

In [18]:
# Revenue by category (multi-table join)
con.execute("""
    SELECT 
        cat.product_category_name_english AS category,
        COUNT(DISTINCT oi.order_id) AS num_orders,
        SUM(oi.price) AS total_revenue,
        ROUND(AVG(oi.price), 2) AS avg_item_price
    FROM order_items oi
    INNER JOIN products p ON oi.product_id = p.product_id
    INNER JOIN categories cat ON p.product_category_name = cat.product_category_name
    GROUP BY cat.product_category_name_english
    ORDER BY total_revenue DESC
    LIMIT 10
""").df()

,category,num_orders,total_revenue,avg_item_price
0,health_beauty,8836,1258681.34,130.16
1,watches_gifts,5624,1205005.68,201.14
2,bed_bath_table,9417,1036988.68,93.30
3,sports_leisure,7720,988048.97,114.34
4,computers_accessories,6689,911954.32,116.51
5,furniture_decor,6449,729762.49,87.56
6,cool_stuff,3632,635290.85,167.36
7,housewares,5884,632248.66,90.79
8,auto,3897,592720.11,139.96
9,garden_tools,3518,485256.46,111.63


**What happened:**
1. First join: `order_items` → `products` (via `product_id`)
2. Second join: `products` → `categories` (via `product_category_name`)
3. Aggregated: SUM(price) to get total revenue per category

**Business insight:** Health & beauty, watches, and bed/bath products are top revenue generators.

---

### Example 4: The Implicit INNER JOIN

**Important note:** The keyword `INNER` is optional. `JOIN` alone defaults to `INNER JOIN`.

These two queries are **identical:**

In [13]:
# Explicit INNER JOIN
result1 = con.execute("""
    SELECT COUNT(*) as count
    FROM orders o
    INNER JOIN customers c ON o.customer_id = c.customer_id
""").df()

# Implicit INNER JOIN (just "JOIN")
result2 = con.execute("""
    SELECT COUNT(*) as count
    FROM orders o 
    JOIN customers c ON o.customer_id = c.customer_id // here we define the alias for customers c and instru
""").df()

print(f"Explicit INNER JOIN: {result1['count'][0]:,} rows")
print(f"Implicit INNER JOIN: {result2['count'][0]:,} rows")
print("\n✅ Both return the same results!")

Explicit INNER JOIN: 99,441 rows
Implicit INNER JOIN: 99,441 rows

✅ Both return the same results!


**Best practice:** Be explicit in your code. Write `INNER JOIN` so readers know your intent.

---

### LEFT JOIN: Keep All Records from Left Table (⏱️ 10 minutes)

> **LEFT JOIN returns ALL rows from the left table, with matches from the right table (or NULL if no match)**

![LEFT JOIN Venn Diagram](../../references/images/left_join.svg)

**Business use case:** "Show me ALL orders, even if they don't have reviews"

---


### Example 5: Basic LEFT JOIN

**Business question:** "Show me all orders with their review scores (if they have one)"

In [14]:
# Demonstrate LEFT JOIN: Showing both matched and unmatched rows

print("=" * 70)
print("Example A: Orders WITHOUT reviews (LEFT JOIN shows NULLs)")
print("=" * 70)

without_reviews = con.execute("""
    SELECT 
        o.order_id,
        o.order_status,
        r.review_score,
        r.review_id
    FROM orders o
    LEFT JOIN reviews r ON o.order_id = r.order_id
    WHERE r.review_id IS NULL
    LIMIT 5
""").df()

print(without_reviews.to_string(index=False))

print("\n" + "=" * 70)
print("Example B: Orders WITH reviews (LEFT JOIN shows matches)")
print("=" * 70)

with_reviews = con.execute("""
    SELECT 
        o.order_id,
        o.order_status,
        r.review_score,
        r.review_id
    FROM orders o
    LEFT JOIN reviews r ON o.order_id = r.order_id
    WHERE r.review_id IS NOT NULL
    LIMIT 5
""").df()

print(with_reviews.to_string(index=False))

print("\n✅ Key point: LEFT JOIN keeps ALL orders from the left table.")
print("   When no review exists, review columns show NULL (displayed as NaN).")

Example A: Orders WITHOUT reviews (LEFT JOIN shows NULLs)
                        order_id order_status  review_score review_id
a9b151f0c0471d9b2534fa73c9c0e123    delivered          <NA>      None
5e8d0f2f1e06e715aee3eefe4c175e52    delivered          <NA>      None
e639eb028398f98b7958207b2a45684c    delivered          <NA>      None
5e42950b6aa90d698e75af4e41641386    delivered          <NA>      None
ec9f9865708006552e26ff7a0e56738b    delivered          <NA>      None

Example B: Orders WITH reviews (LEFT JOIN shows matches)
                        order_id order_status  review_score                        review_id
e481f51cbdc54678b7cc49136f2d6af7    delivered             4 a54f0611adc9ed256b57ede6b6eb5114
53cdb2fc8bc7dce0b6741e2150273451    delivered             4 8d5266042046a06655c8db133d120ba5
47770eb9100c2d0c44946d9cf07ec65d    delivered             5 e73b67b67587f7644d5bd1a52deb1b01
949d5b44dbf5de918fe9c16f97b45f8a    delivered             5 359d03e676b3c069f62cadba8dd3f6e8

**What happened:**

**Example A (orders WITHOUT reviews):**
- `review_score` and `review_id` are **NULL** (shown as `NaN` in pandas)
- LEFT JOIN kept the order even though no matching review exists
- This is the key difference from INNER JOIN (which would drop these rows)

**Example B (orders WITH reviews):**
- All columns are populated with real data
- LEFT JOIN found matches in the reviews table

**Why we filtered:** 
Only 0.77% of orders lack reviews in this dataset, so a random sample rarely shows NULLs. We filtered explicitly (WHERE IS NULL / IS NOT NULL) to demonstrate both cases clearly.

**In practice:** You'd use LEFT JOIN without filtering to get ALL orders (both with and without reviews), then handle NULLs appropriately in your analysis.

---

### Example 6: The Anti-Join Pattern (Finding Unmatched Records)

**Business question:** "Which orders DON'T have reviews yet?"

**Strategy:** LEFT JOIN + filter for NULLs in the right table

In [ ]:
# Find orders without reviews (anti-join pattern)
con.execute("""
    SELECT 
        o.order_id,
        o.order_status,
        o.order_purchase_timestamp
    FROM orders o
    LEFT JOIN reviews r ON o.order_id = r.order_id
    WHERE r.review_id IS NULL
    LIMIT 10
""").df()

**How it works:**
1. LEFT JOIN keeps all orders
2. Orders without reviews have `NULL` for `r.review_id`
3. `WHERE r.review_id IS NULL` filters to only unmatched orders

**Business insight:** These are orders that need follow-up for customer feedback!

Let's see how many:

In [ ]:
# Count unreviewed orders
con.execute("""
    SELECT 
        COUNT(*) AS unreviewed_orders,
        ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM orders), 2) AS pct_unreviewed
    FROM orders o
    LEFT JOIN reviews r ON o.order_id = r.order_id
    WHERE r.review_id IS NULL
""").df()

**Result:** About 0.77% of orders lack reviews. This is valuable data quality insight!

---

### Example 7: LEFT JOIN with Aggregation

**Business question:** "Show me average review score by order status (including orders without reviews)"

In [ ]:
# Average review score by order status
con.execute("""
    SELECT 
        o.order_status,
        COUNT(DISTINCT o.order_id) AS total_orders,
        COUNT(r.review_id) AS reviewed_orders,
        ROUND(AVG(r.review_score), 2) AS avg_review_score
    FROM orders o
    LEFT JOIN reviews r ON o.order_id = r.order_id
    GROUP BY o.order_status
    ORDER BY total_orders DESC
""").df()

**Key observations:**
- `COUNT(DISTINCT o.order_id)` counts all orders (including those without reviews)
- `COUNT(r.review_id)` only counts orders WITH reviews (NULL excluded automatically)
- `AVG(r.review_score)` automatically excludes NULLs

**Business insight:** Delivered orders have excellent ratings (~4.1 stars), but many orders are still in process or canceled.

---

### RIGHT JOIN and FULL JOIN: Quick Mention (⏱️ 2 minutes)

### RIGHT JOIN

> **RIGHT JOIN is just a LEFT JOIN with tables flipped**

```sql
-- These are equivalent:
SELECT * FROM orders o RIGHT JOIN customers c ON o.customer_id = c.customer_id
SELECT * FROM customers c LEFT JOIN orders o ON c.customer_id = o.customer_id
```

**Industry reality:** RIGHT JOIN is rarely used. People just flip the table order and use LEFT JOIN instead.

**Our dataset:** No natural RIGHT JOIN examples (all products have orders, all sellers have sales).

---

### FULL OUTER JOIN

> **FULL OUTER JOIN returns ALL rows from BOTH tables, with NULLs where no match exists**

```sql
-- Keep everything from both tables
SELECT * FROM orders o FULL OUTER JOIN reviews r ON o.order_id = r.order_id
```

**When you'd use this:** Rare! Typically when reconciling two sources and want to see what's in A but not B, and what's in B but not A.

**Business example:** Comparing two inventory systems to find discrepancies.

**For today:** Focus on INNER and LEFT—they cover 95% of real-world use cases.

---

## Part 3: Modern SQL Pattern - CTEs (⏱️ 6 minutes)

### Common Table Expressions (WITH clause)

> **CTEs let you break complex queries into readable, named steps**

**The problem:** Nested subqueries are hard to read and debug.

**The solution:** CTEs ("WITH" clause) - temporary named result sets.

**Think of it as:** "First, calculate this intermediate result and give it a name. Then use it."

---

### Example 8: CTE for Multi-Step Analysis

**Business question:** "Show me sellers with total revenue > $10,000"

**Without CTE (messy nested query):**

In [ ]:
# Without CTE - harder to read
con.execute("""
    SELECT *
    FROM (
        SELECT 
            s.seller_id,
            s.seller_state,
            SUM(oi.price) AS total_revenue
        FROM order_items oi
        INNER JOIN sellers s ON oi.seller_id = s.seller_id
        GROUP BY s.seller_id, s.seller_state
    ) AS seller_revenue
    WHERE total_revenue > 10000
    ORDER BY total_revenue DESC
    LIMIT 10
""").df()

**With CTE (much cleaner):**

In [ ]:
# With CTE - clear two-step logic
con.execute("""
    WITH seller_revenue AS (
        SELECT 
            s.seller_id,
            s.seller_state,
            SUM(oi.price) AS total_revenue
        FROM order_items oi
        INNER JOIN sellers s ON oi.seller_id = s.seller_id
        GROUP BY s.seller_id, s.seller_state
    )
    SELECT *
    FROM seller_revenue
    WHERE total_revenue > 10000
    ORDER BY total_revenue DESC
    LIMIT 10
""").df()

**What happened:**
1. **Step 1 (WITH clause):** Calculate revenue per seller → name it `seller_revenue`
2. **Step 2 (main query):** Filter `seller_revenue` for high earners

**Why this is better:**
- ✅ Reads top-to-bottom (like a recipe)
- ✅ Each step is clearly named
- ✅ Easier to debug (test each CTE separately)
- ✅ Can chain multiple CTEs

**This is how professionals write SQL.** Use CTEs for complex queries!

---

### Example 9: Multiple CTEs

**Business question:** "Compare seller performance in São Paulo vs other states"

You can chain multiple CTEs:

In [22]:
# Multiple CTEs - building up complexity
con.execute("""
    WITH seller_revenue AS (
        -- Step 1: Calculate revenue per seller
        SELECT 
            s.seller_id,
            s.seller_state,
            SUM(oi.price) AS total_revenue
        FROM order_items oi
        INNER JOIN sellers s ON oi.seller_id = s.seller_id
        GROUP BY s.seller_id, s.seller_state
    ),
    state_summary AS (
        -- Step 2: Aggregate by state
        SELECT 
            seller_state,
            COUNT(*) AS num_sellers,
            ROUND(AVG(total_revenue), 2) AS avg_revenue_per_seller,
            ROUND(SUM(total_revenue), 2) AS total_state_revenue
        FROM seller_revenue
        GROUP BY seller_state
    )
    -- Step 3: Compare SP to others
    SELECT 
        CASE 
            WHEN seller_state = 'SP' THEN 'São Paulo'
            ELSE 'Other States'
        END AS region,
        SUM(num_sellers) AS total_sellers,
        ROUND(AVG(avg_revenue_per_seller), 2) AS avg_revenue
    FROM state_summary
    GROUP BY region
    ORDER BY avg_revenue DESC
""").df()

,region,total_sellers,avg_revenue
0,Other States,1246.0,4923.97
1,São Paulo,1849.0,4734.12


**Three clear steps:**
1. Calculate seller-level revenue
2. Aggregate to state level
3. Compare SP vs others

**Business insight:** São Paulo sellers perform similarly to sellers in other states on average.

---

## 🛑 BREAK POINT - Questions?

**So far we've covered:**
- ✅ ERDs and PK/FK relationships
- ✅ INNER JOIN (matching records only)
- ✅ LEFT JOIN (all from left table)
- ✅ Anti-join pattern (finding unmatched records)
- ✅ CTEs for readable multi-step queries

**Next up:**
- ⚠️ The dangers: duplicate inflation and aggregation mistakes
- 🎯 The grand finale: putting it all together with window functions

---

**Take a moment to ask questions before we continue!**

---

## Part 4: The Dangers (⏱️ 12 minutes)

### Duplicate Inflation: The Silent Killer (⏱️ 7 minutes)

> **⚠️ WARNING: Joins can multiply your rows if you're not careful!**

**The problem:** When you join tables with one-to-many relationships, rows get duplicated.

**Why this matters:** 
- Your counts will be WRONG
- Your aggregations will be WRONG
- You'll report incorrect business metrics

**This is the #1 mistake beginners make with joins.**

---

### Example 10: Demonstrating the Problem

**Business question:** "How many orders do we have?"

**Let's count orders three different ways:**

In [ ]:
# Method 1: Count orders directly (correct)
direct_count = con.execute("""
    SELECT COUNT(*) AS order_count
    FROM orders
""").df()

print("Method 1 - Direct count from orders table:")
print(direct_count)
print()

In [ ]:
# Method 2: Count after joining to order_items (WRONG!)
wrong_count = con.execute("""
    SELECT COUNT(*) AS order_count
    FROM orders o
    INNER JOIN order_items oi ON o.order_id = oi.order_id
""").df()

print("Method 2 - Count after joining to order_items (WRONG):")
print(wrong_count)
print("\n⚠️ This is INFLATED because orders with multiple items are counted multiple times!")
print()

In [ ]:
# Method 3: Count DISTINCT after joining (correct)
correct_count = con.execute("""
    SELECT COUNT(DISTINCT o.order_id) AS order_count
    FROM orders o
    INNER JOIN order_items oi ON o.order_id = oi.order_id
""").df()

print("Method 3 - Count DISTINCT after joining (CORRECT):")
print(correct_count)
print("\n⚠️ DISTINCT prevents double-counting, but notice we still lost orders!")

**What happened:**
- **Direct count:** 99,441 orders ✅
- **After join:** 112,650 "orders" ❌ (this is actually counting items!)
- **DISTINCT count:** 98,666 orders ⚠️ (wait, where did 775 orders go?)

**Why the inflation?**
- `orders` has 99,441 rows (one per order)
- `order_items` has 112,650 rows (multiple items per order)
- When you join them, orders with 2 items appear twice!

**Important:** Using `COUNT(DISTINCT ...)` prevents counting the same order multiple times, which is good! BUT notice we still get **98,666 instead of 99,441**. We lost **775 orders** because **INNER JOIN drops orders that don't have items** in `order_items`.

**This is a data quality issue** (orphaned orders without items) that `DISTINCT` can't fix. To preserve ALL orders while avoiding duplication, we'd need:
- `LEFT JOIN` (keeps all orders, even without items) + `DISTINCT` (prevents double-counting)

**The lesson:** `DISTINCT` solves duplicate counting, but doesn't solve data loss from INNER JOIN. Always check your row counts at each step!

---

### Example 11: Seeing the Duplication

Let's look at a specific order with multiple items:

In [ ]:
# Find an order with multiple items
multi_item_order = con.execute("""
    SELECT 
        o.order_id,
        o.order_status,
        oi.order_item_id,
        oi.product_id,
        oi.price
    FROM orders o
    INNER JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.order_id IN (
        SELECT order_id 
        FROM order_items 
        GROUP BY order_id 
        HAVING COUNT(*) > 1
        LIMIT 1
    )
""").df()

print("One order, multiple items (order gets duplicated):")
multi_item_order

**See?** The same `order_id` appears multiple times! 

If you count rows, you're counting items, not orders.

**The fix: Always use `COUNT(DISTINCT ...)` when counting after joins across one-to-many relationships.**

---

### Aggregating Correctly After Joins (⏱️ 5 minutes)

> **Key principle: Aggregate at the correct GRAIN**

**Grain = the level of detail in your result**

**Example questions with different grains:**
- "Revenue per order" → Order grain (one row per order)
- "Revenue per product" → Product grain (one row per product)
- "Revenue per seller per state" → Seller-state grain (one row per seller-state combo)

**Your GROUP BY clause defines the grain!**

---

### Example 12: Correct Aggregation

**Business question:** "What's the total revenue per order?"

**Grain: One row per order**

In [15]:
# Revenue per order (correct grain)
con.execute("""
    SELECT 
        o.order_id,
        o.order_status,
        COUNT(oi.order_item_id) AS num_items,
        ROUND(SUM(oi.price), 2) AS total_revenue
    FROM orders o
    INNER JOIN order_items oi ON o.order_id = oi.order_id
    GROUP BY o.order_id, o.order_status
    ORDER BY total_revenue DESC
    LIMIT 10
""").df()

,order_id,order_status,num_items,total_revenue
0,03caa2c082116e1d31e67e9ae3700499,delivered,8,13440.0
1,736e1922ae60d0d6a89247b851902527,delivered,4,7160.0
2,0812eb902a67711a1cb742b3cdaa65ae,delivered,1,6735.0
3,fefacc66af859508bf1a7934eab1e97f,delivered,1,6729.0
4,f5136e38d1a14a4dbd87dff67da82701,delivered,1,6499.0
5,2cc9089445046817a7539d90805e6e5a,delivered,6,5934.6
6,a96610ab360d42a2e5335a3998b4718a,delivered,1,4799.0
7,199af31afc78c699f0dbf71fb178d4d4,delivered,1,4690.0
8,b4c4b76c642808cbe472a32b86cddc95,canceled,2,4599.9
9,8dbc85d1447242f3b127dda390d56e19,delivered,1,4590.0


**Why this works:**
- `GROUP BY o.order_id` collapses multiple items back to one row per order
- `SUM(oi.price)` adds up all items in each order
- Result: One row per order ✅

**Business insight:** The highest-value orders have 10-20 items each and generate $5K-13K in revenue.

---

### Example 13: Multi-Level Aggregation

**Business question:** "What's the average order value per seller state?"

**Strategy:** 
1. First, calculate order value (order grain)
2. Then, average by seller state (state grain)

**This is perfect for a CTE!**

In [ ]:
# Multi-level aggregation with CTE
con.execute("""
    WITH order_revenue AS (
        -- Step 1: Calculate revenue per order
        SELECT 
            o.order_id,
            s.seller_state,
            SUM(oi.price) AS order_value
        FROM orders o
        INNER JOIN order_items oi ON o.order_id = oi.order_id
        INNER JOIN sellers s ON oi.seller_id = s.seller_id
        GROUP BY o.order_id, s.seller_state
    )
    -- Step 2: Average by state
    SELECT 
        seller_state,
        COUNT(DISTINCT order_id) AS num_orders,
        ROUND(AVG(order_value), 2) AS avg_order_value
    FROM order_revenue
    GROUP BY seller_state
    ORDER BY avg_order_value DESC
    LIMIT 10
""").df()

**Key technique:**
- CTE aggregates at order grain first
- Main query aggregates at state grain second
- Clear two-step logic prevents mistakes

**Business insight:** Seller states have similar average order values (~$130-160), but volumes differ dramatically.

---

## Part 5: The Grand Finale - Everything Together (⏱️ 7 minutes)

### 🎯 Synthesis Example: Day 1 + Day 2 Combined

> **"Top 3 sellers per state by revenue"**

**This query combines EVERYTHING we've learned:**
- ✅ **Multi-table INNER JOIN** (today)
- ✅ **CTE** for readable structure (today)
- ✅ **GROUP BY + aggregation** (Day 1 Block B)
- ✅ **ROW_NUMBER() OVER (PARTITION BY...)** (Day 1 Block B)
- ✅ **Filtering on window function result** (Day 1 Block B)

**This is the kind of query you'll write constantly in analytics roles.**

Let's build it step by step:

---

In [16]:
# The Grand Finale: Top 3 sellers per state
con.execute("""
    WITH seller_revenue AS (
        -- Step 1: Calculate total revenue per seller
        SELECT 
            s.seller_id,
            s.seller_state,
            s.seller_city,
            COUNT(DISTINCT oi.order_id) AS num_orders,
            ROUND(SUM(oi.price), 2) AS total_revenue
        FROM order_items oi
        INNER JOIN sellers s ON oi.seller_id = s.seller_id
        GROUP BY s.seller_id, s.seller_state, s.seller_city
    ),
    ranked_sellers AS (
        -- Step 2: Rank sellers within each state
        SELECT 
            seller_id,
            seller_state,
            seller_city,
            num_orders,
            total_revenue,
            ROW_NUMBER() OVER (
                PARTITION BY seller_state 
                ORDER BY total_revenue DESC
            ) AS rank_in_state
        FROM seller_revenue
    )
    -- Step 3: Filter to top 3 per state
    SELECT 
        seller_state,
        rank_in_state,
        seller_city,
        num_orders,
        total_revenue
    FROM ranked_sellers
    WHERE rank_in_state <= 3
    ORDER BY seller_state, rank_in_state
    LIMIT 30
""").df()

,seller_state,rank_in_state,seller_city,num_orders,total_revenue
0,AC,1,rio branco,1,267.00
1,AM,1,manaus,3,1177.00
2,BA,1,lauro de freitas,358,222776.05
3,BA,2,guanambi,21,17522.00
4,BA,3,salvador,62,15048.28
5,CE,1,fortaleza,14,7846.00
6,CE,2,fortaleza,16,6384.00
7,CE,3,caucaia,27,2777.72
8,DF,1,brasilia,144,18729.64
9,DF,2,brasilia,87,10505.10


### Breaking Down the Magic

**Step 1: Calculate seller revenue**
```sql
WITH seller_revenue AS (
    SELECT seller_id, seller_state, SUM(price) AS total_revenue
    FROM order_items JOIN sellers ...
    GROUP BY seller_id, seller_state
)
```
- **INNER JOIN** combines items with sellers
- **GROUP BY** aggregates to seller grain (one row per seller)
- **SUM** calculates total revenue

**Step 2: Rank within states**
```sql
ranked_sellers AS (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY seller_state ORDER BY total_revenue DESC)
    FROM seller_revenue
)
```
- **ROW_NUMBER()** assigns ranks (1, 2, 3...)
- **PARTITION BY seller_state** restarts ranking for each state
- **ORDER BY total_revenue DESC** ranks by revenue (highest first)

**Step 3: Filter to top 3**
```sql
SELECT * FROM ranked_sellers WHERE rank_in_state <= 3
```
- Keep only ranks 1, 2, 3 per state

---

### Why This Query is Beautiful

**Readability:**
- Three clear steps, each with a purpose
- Named CTEs explain what each step does
- Easy to debug (test each CTE independently)

**Power:**
- Answers complex business question in ~20 lines
- Combines multiple advanced techniques
- Scales to millions of rows efficiently

**Business value:**
- "Who are our top performers in each region?"
- Informs regional sales strategies
- Identifies high-value partners

**This is analytics in the real world.** 🎯

---

## Part 6: Wrap-Up & Quick Tips (⏱️ 3 minutes)

### Key Takeaways

**Join Types:**
- **INNER JOIN** - Only matching rows (most common)
- **LEFT JOIN** - All from left table + matches from right
- **Anti-join pattern** - LEFT JOIN + IS NULL (find unmatched)
- RIGHT/FULL - Exist but rarely used

**Modern Patterns:**
- **CTEs** - Break complex queries into readable steps
- **Window functions + joins** - Powerful combo for ranking/analytics

**Critical Warnings:**
- ⚠️ **Duplicate inflation** - Use COUNT(DISTINCT) after joins
- ⚠️ **Aggregation grain** - GROUP BY defines your result grain
- ⚠️ **NULL behavior** - Remember LEFT JOIN creates NULLs

---

### Quick Tips for Writing Joins

**1. Start simple, build complexity**
- Write the join first, verify row counts
- Add GROUP BY and aggregations second
- Test at each step

**2. Always check row counts**
```sql
-- Before join
SELECT COUNT(*) FROM table1;  -- 100 rows

-- After join  
SELECT COUNT(*) FROM table1 JOIN table2 ...;  -- 150 rows?
-- Why the increase? One-to-many relationship!
```

**3. Use CTEs for multi-step logic**
- Makes queries readable
- Easier to debug
- Self-documenting

**4. Name your tables with aliases**
```sql
-- Good
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id

-- Bad
FROM orders
JOIN customers ON orders.customer_id = customers.customer_id
```

**5. When in doubt, use LEFT JOIN**
- Safer than INNER (won't lose data silently)
- Shows you which records don't match
- Can always filter NULLs later if needed

---

## Next: In-Class Exercise

**You'll now practice these concepts with a real business scenario:**

- Paula Costa, VP of Marketplace Operations at Olist, needs insights for a board meeting
- You'll write queries to answer her business questions
- Apply INNER JOIN, LEFT JOIN, CTEs, and aggregations
- Deliver stakeholder-ready analysis

**Open:** `day2_exercise_joins.ipynb`

**You have 25 minutes for the first 3 queries. Let's go!**

---